In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
    !pip install -q transformers
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/MyDrive/gan_bert_for_cpv')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.3 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
from transformers import logging
logging.set_verbosity_error()
import torch
from create_dataset import CreateDataset
from process_data import ProcessData
from utils import *
from torch.utils.data import DataLoader
from run_model import RunModel
from models import BertGenerator, BertDiscriminator,HiddenClassifier1
from transformers import AutoTokenizer, AutoModel, AutoConfig

In [3]:
label_column = "afdeling"
text_column = "Korte beschrijving aanbesteding"
input_df = pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_dutch_ted1.csv")

In [4]:
vc = input_df[label_column].value_counts()
down_sample_vc = vc[vc.values < 1500]
input_df = input_df[input_df[label_column].isin(down_sample_vc.index)]

In [5]:
down_sample_vc

Furniture (incl. Meubelen (m.i.v. kantoormeubelen), inrichtingsartikelen, huishoudelijke apparaten (uitgez. verlichting) en schoonmaakproducten    1450
Medische apparatuur, farmaceutische artikelen en artikelen voor lichaamsverzorging                                                                 1411
Aardolieproducten, brandstof, elektriciteit en andere energiebronnen                                                                               1112
Vervoersdiensten (uitg. vervoer van afval)                                                                                                          982
Diensten voor onderwijs en opleiding                                                                                                                893
Radio-, televisie-, communicatie-, telecommunicatietoestellen en aanverwante apparatuur                                                             869
Overige gemeenschaps-,  sociale en persoonlijke diensten                                

In [6]:
running_device = torch.device("cuda")

In [7]:
prd = ProcessData(input_df, label_column, text_column, sample_num=2000, aug_limit=1500,test_per_label_num=30, drop_limit=100,
                  balanced_test=True)
test_df, suf_df, insuf_df, label_encoder, onehot = prd.process()
value_counts = insuf_df[label_column].value_counts()
model_runner = RunModel(running_device)

After removing duplicate sentence: 16897
After drop cpv with low observations we have : 16681
The num of class is  28
The number of total data is:  16681
Total number of test data is:  840
Total number of train data is:  15841


In [8]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
config = AutoConfig.from_pretrained("GroNLP/bert-base-dutch-cased", output_hidden_states=True)
bert = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased", config=config).to(running_device)

In [11]:
cd = CreateDataset(text_column, label_column, label_encoder, onehot, 1, running_device, max_length=100)
test_dataset = cd.create(test_df)
# suf_dataset = cd.create(suf_df)
test_dl = DataLoader(test_dataset, shuffle=True, batch_size=20)
insuf_ds_dict = cd.create_label_dataset(insuf_df)
insuf_ds_list = list(insuf_ds_dict.values())
original_insuf_dataset = torch.utils.data.ConcatDataset(insuf_ds_list)
# original_train_dataset = torch.utils.data.ConcatDataset([suf_dataset, original_insuf_dataset])

Getting bert hidden states from layer: 1: 100%|██████████| 641/641 [00:07<00:00, 84.10it/s]


In [ ]:
acc_lists=[]
f1_lists=[]
precision_lists=[]
recall_lists=[]
for i in range(1,15):
  batch_size = 20
  gan_aug_datasets = []
  copy_aug_datasets=[]
  for label in insuf_ds_dict:
      classifier = HiddenClassifier1(prd.num_class, bert)
      sub_ds = insuf_ds_dict[label]
      generator = BertGenerator(bert)
      discriminator = BertDiscriminator(bert)
      enrich_num = len(sub_ds)*i
      enrich_batch = int(enrich_num / batch_size)*2
      source_train = DataLoader(sub_ds, shuffle=True, batch_size=20)
      generator, discriminator = model_runner.generator_trainer(generator, discriminator,
                                    source_train, source_train,
                                    batch_num=20, n_epochs=20, 
                                    glr=2e-6, dlr=2e-6)
      one_data, one_label = sub_ds[0]
      size = (batch_size, one_data.size(0), one_data.size(1))
      gan_aug_dataset = model_runner.generate_aug_data(generator, discriminator, 
                                one_label, size=size,batch_num=enrich_batch)
      print(len(sub_ds))
      print(len(gan_aug_dataset))
      gan_aug_datasets.append(gan_aug_dataset)
  gan_aug_train_dataset = torch.utils.data.ConcatDataset(gan_aug_datasets + [original_insuf_dataset])
  gan_train_dl = DataLoader(gan_aug_train_dataset, shuffle=True, batch_size=20)
  classifier = model_runner.train_classifier(gan_train_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
  predictions, labels = model_runner.predict(test_dl, classifier)
  acc, f1, precision, recal=evaluate(labels, predictions)
  acc_lists.append(acc)
  f1_lists.append(f1)
  precision_lists.append(precision)
  recall_lists.append(recal)

0 discriminator loss:  1.3688751101493835 generator loss : 0.0786209209356457 real acc1:  1.0 fake middle confidence: 0.513299028078715
1 discriminator loss:  1.3555648446083068 generator loss : 0.2326526664197445 real acc1:  1.0 fake middle confidence: 0.5179093976815542
2 discriminator loss:  1.348809725046158 generator loss : 0.35757511109113693 real acc1:  1.0 fake middle confidence: 0.5252680083115896
3 discriminator loss:  1.3440032958984376 generator loss : 0.44733974933624265 real acc1:  1.0 fake middle confidence: 0.5337958931922913
4 discriminator loss:  1.3443944156169891 generator loss : 0.5036225914955139 real acc1:  1.0 fake middle confidence: 0.5425793627897898
5 discriminator loss:  1.343074083328247 generator loss : 0.5371995598077774 real acc1:  1.0 fake middle confidence: 0.5498922169208527
6 discriminator loss:  1.3450452148914338 generator loss : 0.5532392174005508 real acc1:  1.0 fake middle confidence: 0.5556051433086395
7 discriminator loss:  1.343014270067215 g

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.658
F1: 0.622
Precision: 0.615
Recall: 0.658
0 discriminator loss:  1.3769548535346985 generator loss : 0.08711939807981253 real acc1:  0.0 fake middle confidence: 0.4776208847761154
1 discriminator loss:  1.3655374705791474 generator loss : 0.2574048139154911 real acc1:  0.20000000049670538 fake middle confidence: 0.4831082373857498
2 discriminator loss:  1.3562924325466157 generator loss : 0.39577739387750627 real acc1:  0.949999988079071 fake middle confidence: 0.48894256850083667
3 discriminator loss:  1.3496676921844482 generator loss : 0.49587358832359313 real acc1:  1.0 fake middle confidence: 0.49592726429303485
4 discriminator loss:  1.3439685702323914 generator loss : 0.5614131152629852 real acc1:  1.0 fake middle confidence: 0.5029633045196533
5 discriminator loss:  1.340635848045349 generator loss : 0.6004695743322372 real acc1:  1.0 fake middle confidence: 0.5097575088342031
6 discriminator loss:  1.3344038367271422 generator loss : 0.623406144976616 real acc1

In [ ]:
import matplotlib.pyplot  as plt
plt.plot(acc_lists,label="ACC")
plt.plot(f1_lists,label="f1 score")
plt.plot(precision_lists,label="Precision")
plt.plot(recall_lists,label="Recall")
plt.title("Data Augmentation for different layers")
plt.legend()
plt.show()

In [ ]:
print("acc",acc_lists)
print("f1",f1_lists)
print("precision",precision_lists)
print("recall",recall_lists)

In [ ]:
from google. colab import runtime 
runtime. unassign()

# Gan enriched data

In [ ]:
copy_aug_insuf_dataset = []
for label in insuf_ds_dict:
    ds = insuf_ds_dict[label]
    aug_insuf_ds = [ds[i % len(ds)] for i in range(balanced_num)]
    copy_aug_insuf_dataset += aug_insuf_ds

NameError: ignored

In [ ]:
copy_aug_train_dataset = torch.utils.data.ConcatDataset(copy_aug_insuf_dataset + [original_train_dataset])

# Only copy enriched dataset

In [ ]:
copy_train_dl = DataLoader(copy_aug_train_dataset, shuffle=True, batch_size=20)
classifier = HiddenClassifier9(prd.num_class, bert).to(running_device)
classifier = model_runner.train_classifier(copy_train_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
evaluate(labels, predictions, prd.label_encoder, prd.num_class, value_counts)

# Hybrid enriched train dataset; copy+gan augmented

In [ ]:
hybrid_train_dataset = torch.utils.data.ConcatDataset(
    gan_aug_datasets + copy_aug_insuf_dataset + [original_train_dataset])
hybrid_train_dl = DataLoader(hybrid_train_dataset, shuffle=True, batch_size=20)
classifier = HiddenClassifier9(prd.num_class, bert).to(running_device)
classifier = model_runner.train_classifier(hybrid_train_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall = evaluate(labels, predictions, prd.label_encoder, prd.num_class, value_counts)

# Only original dataset training

In [ ]:
original_dl = DataLoader(original_train_dataset, shuffle=True, batch_size=20)
classifier = HiddenClassifier1(prd.num_class, bert).to(running_device)
classifier = model_runner.train_classifier(original_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall = evaluate(labels, predictions)

0 train loss:  2.9717725387587675 val acc:  0.3047619096580006
1 train loss:  1.9207754454576653 val acc:  0.39761905443100704
2 train loss:  1.440107233235131 val acc:  0.5261904895305634
3 train loss:  1.1035037373308876 val acc:  0.5464285817884263
4 train loss:  0.8879791532007772 val acc:  0.5714285834914162
5 train loss:  0.7403410421226357 val acc:  0.5845238211609068
Total ACC: 0.585
F1: 0.525
Precision: 0.491
Recall: 0.585


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
